In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv')
df.head()

,职位,公司,薪资,城市,经验,学历,公司性质,公司规模,职位详情页,公司详情页
0,Python高级开发工程师,上海飞络信息科技,1.6-2.3万,上海,5-7年,大专,民营,50-150人,https://jobs.51job.com/shanghai/149114155.html...,https://jobs.51job.com/all/coB2VWM1cwBj1VNwBlX...
1,Python爬虫开发工程师,万得信息技术,1.5-2.5万·14薪,南京·建邺区,3-4年,本科,民营,1000-5000人,https://jobs.51job.com/nanjing/148852710.html?...,https://jobs.51job.com/all/coUzZdMANmVG1TMlM1V...
2,Python开发工程师,苏州城方信息技术,7千-1.2万,苏州·苏州工业园区,1年,本科,民营,50-150人,https://jobs.51job.com/suzhou/118378024.html?s...,https://jobs.51job.com/all/coAWNdPwNiUWpWNgZlB...
3,Python岗位开发（可实习）7000+,青岛黑臻科技信息,6千-1.2万,青岛,无需经验,NaN,民营,50-150人,https://jobs.51job.com/qingdao/143759490.html?...,https://jobs.51job.com/all/coBWRVMlExDjhVNQNjX...
4,Python,广东盛天体育,1-1.5万,东莞·松山湖区,3-4年,大专,民营,150-500人,https://jobs.51job.com/dongguan/149358666.html...,https://jobs.51job.com/all/coUTNXMV8-BT4BZVA3A...


In [21]:
df['学历'] = df['学历'].fillna('不限学历')
edu_type = df['学历'].value_counts().index.to_list()
edu_num = df['学历'].value_counts().to_list()

In [22]:
from pyecharts import options as opts
from pyecharts.charts import Pie
from pyecharts.faker import Faker
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB
c = (
    Pie()
    .add(
        "",
        [
            list(z)
            for z in zip(edu_type,edu_num)
        ],
        center=["40%", "50%"],
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Python学历要求"),
        legend_opts=opts.LegendOpts(type_="scroll", pos_left="80%", orient="vertical"),
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
)
c.load_javascript()

In [23]:
c.render_notebook()

In [24]:
df['城市'] = df['城市'].str.split('·').str[0]
city_type = df['城市'].value_counts().index.to_list()
city_num = df['城市'].value_counts().to_list()

In [25]:
c = (
    Pie()
    .add(
        "",
        [
            list(z)
            for z in zip(city_type,city_num)
        ],
        center=["40%", "50%"],
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Python招聘城市分布"),
        legend_opts=opts.LegendOpts(type_="scroll", pos_left="80%", orient="vertical"),
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
)
c.render_notebook()

In [26]:
def LowMoney(i):
    if '万' in i:
        low = i.split('-')[0]
        if '千' in low:
            low_num = low.replace('千', '')
            low_money = int(float(low_num) * 1000)
        else:
            low_money = int(float(low) * 10000)
    else:
        low = i.split('-')[0]
        if '元/天' in low:
            low_num = low.replace('元/天', '')
            low_money = int(low_num) * 30
        else:
            low_money = int(float(low) * 1000)
    return low_money
df['最低薪资'] = df['薪资'].apply(LowMoney)

In [27]:
def MaxMoney(j):
    Max = j.split('-')[-1].split('·')[0]
    if '万' in Max and '万/年' not in Max:
        max_num = int(float(Max.replace('万', '')) * 10000)
    elif '千' in Max:
        max_num = int(float(Max.replace('千', '')) * 1000)
    elif '元/天' in Max:
        max_num = int(Max.replace('元/天', ''))  * 30
    else:
        max_num = int((int(Max.replace('万/年', ''))  * 10000) / 12)
    return max_num
df['最高薪资'] = df['薪资'].apply(MaxMoney)

In [28]:
def tranform_price(x):
    if x <= 5000.0:
        return '0~5000元'
    elif x <= 8000.0:
        return '5001~8000元'
    elif x <= 15000.0:
        return '8001~15000元'
    elif x <= 25000.0:
        return '15001~25000元'
    else:
        return '25000以上'

In [29]:
df['最低薪资分级'] = df['最低薪资'].apply(lambda x:tranform_price(x))
price_1 = df['最低薪资分级'].value_counts()
datas_pair_1 = [(i, int(j)) for i, j in zip(price_1.index, price_1.values)]
df['最高薪资分级'] = df['最高薪资'].apply(lambda x:tranform_price(x))
price_2 = df['最高薪资分级'].value_counts()
datas_pair_2 = [(i, int(j)) for i, j in zip(price_2.index, price_2.values)]

In [30]:
pie1 = (
    Pie(init_opts=opts.InitOpts(theme='dark',width='1000px',height='600px'))
    
    .add('', datas_pair_1, radius=['35%', '60%'])
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}:{d}%"))
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="Python工作薪资\n\n最低薪资区间", 
            pos_left='center', 
            pos_top='center',
            title_textstyle_opts=opts.TextStyleOpts(
                color='#F0F8FF', 
                font_size=20, 
                font_weight='bold'
            ),
        )
    )
    .set_colors(['#EF9050', '#3B7BA9', '#6FB27C', '#FFAF34', '#D8BFD8', '#00BFFF', '#7FFFAA'])
)
pie1.render_notebook()

In [31]:
pie1 = (
    Pie(init_opts=opts.InitOpts(theme='dark',width='1000px',height='600px'))
    
    .add('', datas_pair_2, radius=['35%', '60%'])
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}:{d}%"))
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="Python工作薪资\n\n最高薪资区间", 
            pos_left='center', 
            pos_top='center',
            title_textstyle_opts=opts.TextStyleOpts(
                color='#F0F8FF', 
                font_size=20, 
                font_weight='bold'
            ),
        )
    )
    .set_colors(['#EF9050', '#3B7BA9', '#6FB27C', '#FFAF34', '#D8BFD8', '#00BFFF', '#7FFFAA'])
)
pie1.render_notebook() 

In [32]:
exp_type = df['经验'].value_counts().index.to_list()
exp_num = df['经验'].value_counts().to_list()
c = (
    Pie()
    .add(
        "",
        [
            list(z)
            for z in zip(exp_type,exp_num)
        ],
        center=["40%", "50%"],
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="Python招聘经验要求"),
        legend_opts=opts.LegendOpts(type_="scroll", pos_left="80%", orient="vertical"),
    )
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
)
c.render_notebook()

In [33]:
# 按城市分组并计算平均薪资
avg_salary = df.groupby('城市')['最低薪资'].mean()
CityType = avg_salary.index.tolist()
CityNum = [int(a) for a in avg_salary.values.tolist()]
avg_salary_1 = df.groupby('城市')['最高薪资'].mean()
CityType_1 = avg_salary_1.index.tolist()
CityNum_1 = [int(a) for a in avg_salary_1.values.tolist()]

In [34]:
from pyecharts.charts import Bar
# 创建柱状图实例
c = (
    Bar()
    .add_xaxis(CityType)
    .add_yaxis("", CityNum)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各大城市Python低平均薪资"),
        visualmap_opts=opts.VisualMapOpts(
            dimension=1,
            pos_right="5%",
            max_=30,
            is_inverse=True,
        ),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=45))  # 设置X轴标签旋转角度为45度
    )
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markline_opts=opts.MarkLineOpts(
            data=[
                opts.MarkLineItem(type_="min", name="最小值"),
                opts.MarkLineItem(type_="max", name="最大值"),
                opts.MarkLineItem(type_="average", name="平均值"),
            ]
        ),
    )
)

c.render_notebook()

In [35]:
# 创建柱状图实例
c = (
    Bar()
    .add_xaxis(CityType_1)
    .add_yaxis("", CityNum_1)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="各大城市Python高平均薪资"),
        visualmap_opts=opts.VisualMapOpts(
            dimension=1,
            pos_right="5%",
            max_=30,
            is_inverse=True,
        ),
        xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=45))  # 设置X轴标签旋转角度为45度
    )
    .set_series_opts(
        label_opts=opts.LabelOpts(is_show=False),
        markline_opts=opts.MarkLineOpts(
            data=[
                opts.MarkLineItem(type_="min", name="最小值"),
                opts.MarkLineItem(type_="max", name="最大值"),
                opts.MarkLineItem(type_="average", name="平均值"),
            ]
        ),
    )
)

c.render_notebook()

### 结论:
    1. 学历要求基本大专以上
    2. 薪资待遇: 8000-25000 左右
    3. 北上广 薪资偏高一些
### 如何简单实现可视化分析
    1. 通过爬虫采集完整的数据内容 --> 表格 / 数据库
    2. 读取文件内容
    3. 统计每个类目的数据情况
    4. 通过可视化模块: <使用官方文档提供代码模板去实现>

In [18]:
import pandas as pd

In [19]:
# 读取数据
df = pd.read_csv('data.csv')
# 显示前五行数据
df.head()

,职位,公司,薪资,城市,经验,学历,公司性质,公司规模,职位详情页,公司详情页
0,Python高级开发工程师,上海飞络信息科技,1.6-2.3万,上海,5-7年,大专,民营,50-150人,https://jobs.51job.com/shanghai/149114155.html...,https://jobs.51job.com/all/coB2VWM1cwBj1VNwBlX...
1,Python爬虫开发工程师,万得信息技术,1.5-2.5万·14薪,南京·建邺区,3-4年,本科,民营,1000-5000人,https://jobs.51job.com/nanjing/148852710.html?...,https://jobs.51job.com/all/coUzZdMANmVG1TMlM1V...
2,Python开发工程师,苏州城方信息技术,7千-1.2万,苏州·苏州工业园区,1年,本科,民营,50-150人,https://jobs.51job.com/suzhou/118378024.html?s...,https://jobs.51job.com/all/coAWNdPwNiUWpWNgZlB...
3,Python岗位开发（可实习）7000+,青岛黑臻科技信息,6千-1.2万,青岛,无需经验,NaN,民营,50-150人,https://jobs.51job.com/qingdao/143759490.html?...,https://jobs.51job.com/all/coBWRVMlExDjhVNQNjX...
4,Python,广东盛天体育,1-1.5万,东莞·松山湖区,3-4年,大专,民营,150-500人,https://jobs.51job.com/dongguan/149358666.html...,https://jobs.51job.com/all/coUTNXMV8-BT4BZVA3A...


In [20]:
c_type = df['公司性质'].value_counts().index.to_list() # 统计数据类目
c_num = df['公司性质'].value_counts().to_list() # 统计数据个数
c_type

['民营', '已上市', '外资（非欧美）', '合资', '外资（欧美）', '国企', '事业单位']

In [21]:
from pyecharts.charts import Bar # 导入pyecharts里面柱状图
from pyecharts.faker import Faker # 导入随机生成数据
from pyecharts.globals import ThemeType # 主题设置

c = (
    Bar({"theme": ThemeType.MACARONS}) # 主题设置
    .add_xaxis(c_type)  # x轴数据
    .add_yaxis("", c_num) # Y轴数据
    .set_global_opts(
        # 标题显示
        title_opts={"text": "Python招聘企业公司性质分布", "subtext": "民营', '已上市', '外资（非欧美）', '合资', '国企', '外资（欧美）', '事业单位'"}
    )
    # 保存html文件
#     .render("bar_base_dict_config.html")
)
# print(Faker.choose()) # ['小米', '三星', '华为', '苹果', '魅族', 'VIVO', 'OPPO'] 数据类目
# print(Faker.values()) # [38, 54, 20, 85, 71, 22, 38] 数据个数
c.render_notebook() # 直接显示在jupyter上面